# Numpy Memory Array for Fast Image Loading

In [1]:
%load_ext autoreload
%autoreload 2

In [11]:
from pathlib import Path

import numpy as np
from PIL import Image
import cv2

from model.memory_map import MemMapReader, MemMapWriter

## Writer

In [61]:
image_paths = Path('outputs/speed_test').glob('*.png')
mm_writer = MemMapWriter(image_paths, 'outputs/mm/mem_map.dat')
mm_writer.write()
print(mm_writer)

Saving memmap: 100%|██████████| 476/476 [00:00<00:00, 2297.96it/s]


## Reader

In [2]:
mm_reader = MemMapReader('outputs/speed_test3/mm2/mem_map.dat', (256, 256))

In [3]:
len(mm_reader)

476

In [4]:
mm_reader[len(mm_reader) - 1].shape

(256, 256)

### Memory Array

Memory array creates **read-only** numpy array.

In [5]:
%%timeit

for img in mm_reader:
    x = img.shape

345 μs ± 33.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [6]:
%%timeit

for img in mm_reader:
    x = np.copy(img).shape

1.75 ms ± 40.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [7]:
%%timeit

for imgs in mm_reader.iter_windows(4):
    x = (len(imgs), imgs[0].shape)

1.4 ms ± 29.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit

for imgs in mm_reader.iter_windows(4):
    x = np.copy(imgs)
    x = (len(x), x[0].shape)

4.71 ms ± 227 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Pillow

In [9]:
%%timeit

for img_path in Path('outputs/speed_test3').glob('*.png'):
    x = np.array(Image.open(img_path)).shape

111 ms ± 6.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit

image_paths = list(Path('outputs/speed_test3').glob('*.png'))
for i in range(0, len(image_paths), 4):
    xs = [np.array(Image.open(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

114 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### OpenCV

In [12]:
%%timeit

for img_path in Path('outputs/speed_test3').glob('*.png'):
    x = cv2.imread(str(img_path)).shape

110 ms ± 2.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit

image_paths = list(Path('outputs/speed_test3').glob('*.png'))
for i in range(0, len(image_paths), 4):
    xs = [cv2.imread(str(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

108 ms ± 1.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Benchmark Result

Memory array is ~20 times faster than Pillow or OpenCV.